# Computação Evolucionária
## Atividade Prática 1
Amanda Kellen Soares de Pinho - 2017098188

Lucas Araujo Azevedo - 2017104188


Nesta atividade foi feita a implementação de um algoritmo evolucionário para solucionar o problema de N-Rainhas que será explicado ao longo deste trabalho.

# Problema das N-Rainhas

O problema apresentado consiste em posicinar N-Rainhas em um tabuleiro N X N de uma maneira que elas não se coloquem em xeque.

Para tal, foi necessário definir os seguintes itens:

1. Representação de uma solução candidata no espaço de busca do problema;
2. Função de aptidão para avaliação da qualidade da solução obtida;
3. Operadores de variação;
4. Mecanismos de seleção;
5. Condição de Término.



In [2]:
import random as rd
import numpy as np
import math

# 1. Solução Candidata

Primeiramente é iniciada a população que será utilizada ao longo do problema, para tal teremos como default um tabelueiro 
8 X 8 e uma população com 20 soluções candidatas geradas aleatoriamente.

In [3]:
def init_population(_mu:int = 20, n:int = 8):
    "Inicia uma população com o tamanho das peças do tabuleiro e a quantidade de rainhas como parâmetro"
    population = []
    for i in range (_mu):
        population.append(rd.sample(range(n), n))

    return population


# 2. Função de Aptidão

Para verificar a quantidade de cheques da disposião das rainhas apresentadas na população

In [4]:
def fitness_nq(solution):
    """Informa a quantidade de cheques em cada solução"""
    xeques = 0
    for i in range(0,len(solution)):
        for j in range(0,len(solution)):
            if i!=j:
                if i-solution[i] == j-solution[j] or i+solution[i] == j+solution[j]:
                    xeques+=1

    return xeques

# 3. Mecanismo de Seleção

## 3.1 Seleção dos pais
Para a seleção da melhor solução, ou seja, da solução que possui o menor número de cheques, foi ultilizada a função apresentada abaixo.Esta tem como objetivo selecionar 5 soluções aleatoriamente e destas selecionar as 2 melhores soluçõe para que assim estes sejam utilizadas para realizar a reprodução.

In [5]:
def selection(pop: list, n_rainhas: int):
    "Retorna as duas melhores soluções (melhores fitness)"
    
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]

    min_xeques_1 = max(pop_fitness)
    min_xeques_2 = min(pop_fitness)
    position_2 = n_rainhas + 1
    position_1 = n_rainhas + 1

    for i, num_xeques in enumerate(pop_fitness):
        if num_xeques ==  min_xeques_2 and position_2 != i:
            position_2 = i
        elif num_xeques < min_xeques_1:
            min_xeques_1 = num_xeques
            position_1 = i
            
    melhores_2 = [pop[position_1], pop[position_2]]
    return melhores_2

## 3.2 Seleção dos Sobreviventes
A cada nova geração é feita a seleção dos sobreviventes, para tal é percorrido a população original, nesta é selecionada os dois piores resultados (maior quantidade de cheques) e estes são substituídos pela nova geração que foi obtida através do cruzamento de dois indivíduos.

In [6]:
def replacement(offspring_new, pop):
    "Atualiza a lista de população, substituindo os dois piores individuos pelos dois que fazem parte do subset offspring new"
    
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]

    max_xeques_1 = max(pop_fitness)
    max_xeques_2 = min(pop_fitness)
    position_1 = n_rainhas + 1
    position_2 = n_rainhas + 1

    for i, num_xeques in enumerate(pop_fitness):
        if num_xeques ==  max_xeques_1 and position_1 != i:
            position_1 = i
        elif num_xeques > max_xeques_2:
            max_xeques_2 = num_xeques
            position_2 = i
    # print("----------------------------------------------------------------")
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    # print(pop_fitness)
    pop[position_1] = offspring_new[0]
    pop[position_2] = offspring_new[1]
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    # print(pop_fitness)
    return pop

# 4. Operadores de Variação

## 4.1 Crosssover
Essa função é utilizada para a reprodução de novos indivíduos, para isso é necessário informar um par de pais, estes deverão ser os que obtiverem o melhor resultado utilizando a função selection para tal.

In [7]:
def crossover(subset_parents, crossover_rate):
    "Retorna um subset (dois indivíduos filhos) como resultado do cruzamento dos pais"
    len_parents = len(subset_parents[0])
    cut = int(1+len_parents*crossover_rate) # Valor onde irá cortar
    
    # Fazendo a combinação entre os pais para gerar os filhos
    son_1 = subset_parents[0].copy()
    son_2 = subset_parents[1].copy()
    
    son_1_genes = son_1[cut:]
    son_1[cut:] = son_2[cut:]
    son_2[cut:] = son_1_genes
    
    return [son_1, son_2]

## 4.2 Mutação
Esta função seleciona de maneira ramdômica duas posições de rainhas e troca os valores destas posições que foram selecionadas utilizando a probabilidade de 0.8 de mutação.

In [8]:
def mutation(offspring, mutation_rate, n_rainhas):
    "Retorna um subset (dois indivíduos) como resultado da mutação dos filhos"
    size_ind = len(offspring[0])
    quant_mut = int(size_ind*mutation_rate)
    index_mult = np.linspace(0, size_ind-1, quant_mut)
    
    new_springs = []
    for spring in offspring:
        for i in index_mult:
            i = int(i)
            new_number = rd.randint(0, n_rainhas-1)
            spring[i] = new_number
            
        new_springs.append(spring)

    return new_springs

In [9]:
def evalution(pop: list):
    "Retorna o melhor resultado e a média de xeques"
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    fitness_best = min(pop_fitness)
    fitness_mean = np.mean(pop_fitness)
    return fitness_best, fitness_mean

In [10]:
i_geracoes = 0

crossover_rate = 0.5
mutation_rate = 0.8 
n_rainhas = 8
tamanho_tabuleiro = 20
num_epochs = 10
fitness_best = None

list_best_fitness = []
list_mean_fitness = []

pop = init_population(tamanho_tabuleiro, n_rainhas)

print("Melhor Resultado | Resultado Médio")
while i_geracoes <= num_epochs and fitness_best != 0:
    subset_parents = selection(pop, n_rainhas)
    offspring = crossover(subset_parents, crossover_rate)
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    offspring_new = mutation(offspring, mutation_rate, n_rainhas)
    
    pop = replacement(offspring_new, pop)
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    
    fitness_best, fitness_mean = evalution(pop)
    print(f"\t{fitness_best} \t | \t{fitness_mean}")
    list_best_fitness.append(fitness_best)
    list_mean_fitness.append(fitness_mean)
    i_geracoes += 1
    

Melhor Resultado | Resultado Médio
	4 	 | 	8.2
	4 	 | 	8.4
	4 	 | 	7.7
	4 	 | 	7.5
	4 	 | 	7.7
	4 	 | 	7.1
	4 	 | 	7.1
	4 	 | 	6.7
	4 	 | 	6.8
	4 	 | 	6.5
	4 	 | 	6.5


# Conclusão

Conforme podemos observara acima, podemos concluir que os resultados obtivos foram satisfatórios, visto que a cada nova geração a media de cheques reduz, o que é o desejado.